In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import matplotlib as pl
from msa2qubo import *
from qubo2msa import *
import numpy as np

In [ ]:
m2q = Msa2Qubo(input="./seq.fa", output="t2/qmsa.qubo", P=1, verbose=False, delta=2.0, l0=1.0, l1=0.1, l2=10.0)

In [ ]:
m2q.run()

In [ ]:
import math

In [ ]:
X = m2q.bvc.x()
print("seqs=",X)
print("# seqs=",len(X))
nr = 0
ny = 0
nz = 0
for k in range(0,len(X)-1):
    for q in range(k+1, len(X)):
        for i in range(0, len(X[k])):
            for j in range(0, len(X[q])):
                nr+=1
                ny+=1
                nz+=1
print("#R=",nr)
print("#Y=",ny)
print("#Z=",nz)
print("#Additional Vars=",nr+ny+nz)

In [ ]:
numPosVars = m2q.bvc.get_NbPositioningVars(intmode=True)
numGapVars = m2q.bvc.get_NbGapVars(intmode=True)

print(numPosVars)
print(numGapVars)

In [ ]:
e2m = np.zeros((numPosVars+numGapVars+nr+ny+nz,numPosVars+numGapVars+nr+ny+nz))
print(e2m.shape)

In [ ]:
solution_vector = np.zeros((1, numPosVars+numGapVars+nr+ny+nz))

In [ ]:
# Move to the beggining of the matrix where the positions for R,Y and Z are
rMatPos = numPosVars+numGapVars

# Place the R*W variables on the R part of the matrix
pbK = 0
pbQ = 0
posI = 0
posJ = 0
for k in range(0,len(X)-1):
    pbK += len(X[k])
    for q in range(k+1, len(X)):
        pbQ += len(X[q])
        for i in range(0, len(X[k])):
            posI = 0
            posI = pbK+i
            for j in range(0, len(X[q])):
                posJ = 0
                posJ = pbQ+j
                matPos = rMatPos + pbK + pbQ + posI + posJ
                e2m[matPos][matPos] = m2q.bvc.w(i,j,k,q)


In [ ]:
# Place the X*Y, X*Z variables in the matrix multiplied by l_2 and W_ijkq

In [ ]:
yMatPos = (numPosVars+numGapVars)*2
zMatPos = (numPosVars+numGapVars)*3

pbK = 0
pbQ = 0
posI = 0
posJ = 0
for k in range(0,len(X)-1):
    for q in range(k+1, len(X)):
        for i in range(0, len(X[k])):
            # This loop would need to be extended for the binary version
            posI = 0
            posI = pbK+i
            for j in range(0, len(X[q])):
                # This loop would need to be extended for the binary version
                posJ = 0
                posJ = pbQ+j
                YmatPos = yMatPos + pbK + pbQ + posI + posJ
                ZmatPos = zMatPos + pbK + pbQ + posI + posJ
                e2m[pbK+i][YmatPos] = -1. * m2q.bvc.l2() * m2q.bvc.w(i,j,k,q)
                e2m[pbQ+j][YmatPos] = 2. * m2q.bvc.l2() * m2q.bvc.w(i,j,k,q)
                e2m[pbQ+j][ZmatPos] = -1. * m2q.bvc.l2() * m2q.bvc.w(i,j,k,q)
        pbQ += len(X[q])
    pbK += len(X[k])


In [ ]:
# Place the bounds on Y and Z based on the McCormick envelopes

#YunderMin = 0
#YunderMax = X_ki + Rijkq*Lmax - Lmax

#YoverMin = X_ki
#YoverMax = Rijkq*Lmax

# This goes on the integer problem constraints matrix/array

In [ ]:
m = m2q.bvc.m()
nbR = nr
nbY = ny*m
nbZ = nz*m

print("nbR", nbR)
print("nbY", nbY)
print("nbZ", nbZ)

In [ ]:

delta = 1
bVarRPos = m2q.bvc.get_NbPositioningVars() + m2q.bvc.get_NbGapVars()
bVarYPos = bVarRPos + nr # No need to transform R to binary as it only either (0,1)
# Since R is either 0,1 in the following loops need to take that into account...
bVarZPos = bVarYPos + ny*m
matDims = bVarZPos+nz*m
e2bm = np.zeros((matDims, matDims))
pbK = 0
pbQ = 0
posI = 0
posJ = 0
piK = 0
piQ = 0
for k in range(0,len(X)-1):
    for q in range(k+1, len(X)):
        pbQ += len(X[k])*m
        piQ += len(X[k])
        posI = 0
        for i in range(0, len(X[k])):
            for bi in range(0,m):
                posJ = 0
                for j in range(0, len(X[q])):
                    for bj in range(0, m):
                        bRMatPos = bVarRPos
                        bYMatPos = bVarYPos
                        bZMatPos = bVarZPos
                        idx = posI+bi + posJ+bj + pbK+pbQ
                        
                        # -Yijkq*Xki
                        e2bm[pbK+posI+bi][bYMatPos+idx] += -1. #* 2**(bi+bj)
                        # 3d * Yijkq
                        e2bm[bYMatPos+idx][bYMatPos+idx] += 3.*delta #* 2**(bi+bj)
                        # d * Rijqk*Xki
                        e2bm[pbK+posI+bi][bRMatPos+piK+piQ+i+j] += 1.*delta #* 2**(bi+bj)
                        # -2d * Yijkq*Rijkq
                        e2bm[bRMatPos+piK+piQ+i+j][bYMatPos+idx] += -2.*delta #* 2**(bi+bj)
                        # 2d * Yijkq*Xki
                        e2bm[pbK+posI+bi][bYMatPos+idx] += 2.*delta #* 2**(bi+bj)
                        # -4d * Yijkq*Xqj
                        e2bm[pbQ+posJ+bj][bYMatPos+idx] += -4.*delta #* 2**(bi+bj)
                        # Zijkq*Xqj
                        e2bm[pbQ+posJ+bj][bZMatPos+idx] += 1. #* 2**(bi+bj)
                        # -3d * Zijkq
                        e2bm[bZMatPos+idx][bZMatPos+idx] += -3.*delta #* 2**(bi+bj)
                        # -d * Rijkq*Xqj
                        e2bm[pbQ+posJ+bj][bRMatPos+piK+piQ+i+j] += -1.*delta #* 2**(bi+bj)
                        # 2d * Zijqk*Xqj
                        e2bm[pbQ+posJ+bj][bZMatPos+idx] += 2.*delta #* 2**(bi+bj)
                        # 2d * Zijkq*Rijkq
                        e2bm[bRMatPos+piK+piQ+i+j][bZMatPos+idx] += 2.*delta #* 2**(bi+bj)
                    posJ += m
            posI += m
    pbK += len(X[k])*m
    piK += len(X[k])


In [ ]:
print("nX", 0)
print("nG", m2q.bvc.get_NbPositioningVars())
print("nR", bVarRPos)
print("nY", bVarYPos)
print("nZ", bVarZPos)


In [ ]:

G = np.zeros((matDims,matDims,3))
G[e2bm!=0] = [0,0,0]
G[e2bm==0] = [1,1,1]
plt.figure(figsize=(10,10))

#plt.annotate(s="G", xy=((m2q.bvc.get_NbPositioningVars())*0.5,-1))
#plt.annotate(s="G", xy=(matDims,(m2q.bvc.get_NbPositioningVars())*0.5))
plt.axvline(m2q.bvc.get_NbPositioningVars())
plt.axhline(m2q.bvc.get_NbPositioningVars())

plt.axvline(m2q.bvc.get_NbPositioningVars()+m2q.bvc.get_NbGapVars())
plt.axhline(m2q.bvc.get_NbPositioningVars()+m2q.bvc.get_NbGapVars())

plt.annotate(s="R", xy=((bVarRPos+bVarYPos)*0.5,-1))
plt.annotate(s="R", xy=(matDims,(bVarRPos+bVarYPos)*0.5))
plt.axvline(bVarRPos)
plt.axhline(bVarRPos)

plt.annotate(s="Y", xy=((bVarYPos+bVarZPos)*0.5,-1))
plt.annotate(s="Y", xy=(matDims,(bVarYPos+bVarZPos)*0.5))
plt.axvline(bVarYPos)
plt.axhline(bVarYPos)

plt.annotate(s="Z", xy=((bVarZPos+matDims)*0.5,-1))
plt.annotate(s="Z", xy=(matDims+0.11,(bVarZPos+matDims)*0.5))
plt.axvline(bVarZPos)
plt.axhline(bVarZPos)

plt.imshow(G, cmap='Greys', interpolation='nearest')
plt.show()